In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from DiceTowerVision import *
import datetime
cam_id = 1
camera_resolution = (2592,1944)

background_file_template = "Images\\Calibration\\background (*).jpg"
roll_file_template = "Images\\Rolls\\New\\FN#.jpg"

background_images_RGB = get_mactching_images_RGB(background_file_template,log_level=LOG_LEVEL_INFO)
roi_mask = get_roi_mask_from_bg_images(background_images_RGB,range=(0,10), filter=25, dilation=5, log_level=LOG_LEVEL_INFO)

# calibration_file = "Images\\Calibration\\calibration (1).jpg"
# cal_BGR = cv.imread(calibration_file)
# map_x, map_y = calibrate_from_checkerboard(cv.cvtColor(cal_BGR,cv.COLOR_BGR2GRAY),(10,10))

file_template = "Images\\Templates\\DDR# (A) (FV#).jpg"
dieset = DieSet.create_common_die_set_from_images(file_template,names=[4,6,8,10,12,20,100],roi_mask=roi_mask,log_level=LOG_LEVEL_WARN)


In [6]:
motion_history = 5
motion_ratio = 1 - 1/motion_history
motion_detector = cv.createBackgroundSubtractorMOG2(history=motion_history)
fg_detector = cv.createBackgroundSubtractorMOG2()
camera = cv.VideoCapture(cam_id, cv.CAP_DSHOW)
if not camera.isOpened():
    print("Camera is not connected!")
camera.set(cv.CAP_PROP_FRAME_WIDTH, camera_resolution[0])
camera.set(cv.CAP_PROP_FRAME_HEIGHT, camera_resolution[1])



fid = 0
tray_mask = cv.morphologyEx(roi_mask,cv.MORPH_ERODE, get_kernel(5), iterations=50)
fg_threshold = 1.0
motion_start_threshold = 0.1
motion_end_threshold = 0.01
motion_active = False
image_captured = False
cv.destroyAllWindows()

while True:
    fid += 1
    result, image_BGR = camera.read()
    if image_BGR is None:
        print(str(result))
        break
    # image_BGR = cv.remap(image_BGR, map_x, map_y, cv.INTER_LINEAR)
    image_BGR_small = cv.resize(image_BGR,None,fx=0.25,fy=0.25,interpolation=cv.INTER_AREA)
    cv.imshow('Frame',image_BGR_small)

    image_GRAY = cv.bitwise_and(cv.cvtColor(image_BGR, cv.COLOR_BGR2GRAY),tray_mask)
    image_blur = cv.medianBlur(image_GRAY,5)
    image = cv.inRange(image_blur,DieFaceSample.backround_brightness_threshold,255)
    motion_mask = motion_detector.apply(image)
    #cv.imshow('Motion', motion_mask)

    # if fid <= 2*motion_history:
    #     fg_mask = fg_detector.apply(image)
    #     #continue
    # else:
    #     fg_mask = fg_detector.apply(image, learningRate=0)
    fg_mask_small = cv.resize(image, (int(camera_resolution[0]/8), int(camera_resolution[1]/8)))
    #cv.imshow('Foreground', fg_mask_small)
    fg_sum = np.mean(image)
    motion_sum = np.mean(motion_mask)

    if fid > 2*motion_history:
        if fg_sum > fg_threshold:
            if not motion_active:
                if motion_sum > motion_start_threshold:
                    motion_active = True
            if motion_active:
                if motion_sum < motion_end_threshold:
                    motion_active = False
                    if not image_captured:
                        roll_file_name = roll_file_template.replace("FN#",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
                        print(roll_file_name)
                        cv.imwrite(roll_file_name, image_BGR)
                        image_RGB = cv.cvtColor(image_BGR, cv.COLOR_BGR2RGB)
                        dieset.get_die_faces_from_image(image_RGB,face_confidence=0.5, rank_confidence=0.5,roi_mask=roi_mask,log_level=LOG_LEVEL_INFO)
                        # plt.imshow(image_RGB)
                        # plt.show()

                        image_captured = True
                
        else:
            image_captured = False

    
    print("%u: FG=%f ; MOTION=%f ; MO/CAP=%s/%s"%(fid,fg_sum, motion_sum,str(motion_active),str(image_captured)))

    keypress = cv.waitKey(100)
    if keypress == 'q' or keypress == 27:
        break

    
cv.destroyAllWindows()

1: FG=0.000000 ; MOTION=255.000000 ; MO/CAP=False/False
2: FG=0.000000 ; MOTION=0.000000 ; MO/CAP=False/False
3: FG=53.218623 ; MOTION=53.218623 ; MO/CAP=False/False
4: FG=53.219382 ; MOTION=0.250048 ; MO/CAP=False/False
5: FG=53.215384 ; MOTION=0.149796 ; MO/CAP=False/False
6: FG=53.210829 ; MOTION=0.103744 ; MO/CAP=False/False
7: FG=53.210981 ; MOTION=0.086436 ; MO/CAP=False/False
8: FG=53.207995 ; MOTION=0.061892 ; MO/CAP=False/False
9: FG=53.213714 ; MOTION=0.060779 ; MO/CAP=False/False
10: FG=53.231831 ; MOTION=0.056477 ; MO/CAP=False/False
11: FG=53.194433 ; MOTION=0.048937 ; MO/CAP=False/False
12: FG=53.218977 ; MOTION=0.051518 ; MO/CAP=False/False
13: FG=53.217813 ; MOTION=0.047925 ; MO/CAP=False/False
14: FG=53.227631 ; MOTION=0.050000 ; MO/CAP=False/False
15: FG=53.228137 ; MOTION=0.046761 ; MO/CAP=False/False
16: FG=53.221153 ; MOTION=0.041042 ; MO/CAP=False/False
17: FG=53.221811 ; MOTION=0.049544 ; MO/CAP=False/False
18: FG=53.218876 ; MOTION=0.051771 ; MO/CAP=False/False


KeyboardInterrupt: 

: 